In [3]:
from transformers import AutoTokenizer, AutoModel
# from bertviz import head_view, model_view
from datasets import load_dataset

model_name = 'hyunwoongko/kobart'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_attentions=True)
dataset = load_dataset("metamong1/summarization_paper", use_auth_token=True)

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
text = dataset['train'][2]['text'][:100]
title = dataset['train'][2]['title']

encoder_input_ids = tokenizer(text, return_tensors="pt", add_special_tokens=True).input_ids
decoder_input_ids = tokenizer(title, return_tensors="pt", add_special_tokens=True).input_ids

text, title

('본 연구는 최근 20년간 스웨덴 지방자치단체가 경험한 주요 환경정책을 검토하여 거버넌스의 특징이 어떻게 변화하였는지 분석하고 그에 따른 시사점을 도출한 것이다. 구체적인 사례로 지',
 '스웨덴 지역환경정책 거버넌스의 변화와 특징 연구 -지방의제21(LA21), 녹색성장 지역투자프로그램(LIP), 기후변화투자프로그램(Klimp)의 사례-')

In [7]:
# outputs = model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)
outputs = model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)

encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0])

In [8]:
# head_view(
#     cross_attention=outputs.cross_attentions,
#     encoder_tokens=encoder_text,
#     decoder_tokens=decoder_text,
# )
# model_view(
#     cross_attention=outputs.cross_attentions,
#     encoder_tokens=encoder_text,
#     decoder_tokens=decoder_text,
#     display_mode="light",
# )

In [9]:
outputs.cross_attentions[0].shape

torch.Size([1, 16, 41, 39])

In [10]:
import chart_studio
chart_studio.tools.set_credentials_file(username='presto105', api_key='1pCoh8tqWL2KzeKVCHcV')

In [11]:
import pandas as pd
import numpy as np
import scipy as sp
import chart_studio.plotly as py

In [12]:
import chart_studio.plotly as py
import plotly.figure_factory as ff
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv")

table = ff.create_table(df)

In [13]:
import torch
def format_attention(attention, layers=None, heads=None):
    if layers:
        attention = [attention[layer_index] for layer_index in layers]
    squeezed = []
    for layer_attention in attention:
        # 1 x num_heads x seq_len x seq_len
        if len(layer_attention.shape) != 4:
            raise ValueError("The attention tensor does not have the correct number of dimensions. Make sure you set "
                             "output_attentions=True when initializing your model.")
        layer_attention = layer_attention.squeeze(0)
        if heads:
            layer_attention = layer_attention[heads]
        squeezed.append(layer_attention)
    # num_layers x num_heads x seq_len x seq_len
    return torch.stack(squeezed)

In [14]:
len(encoder_text), len(decoder_text)
e = np.array(encoder_text)
d = np.array(decoder_text)

In [15]:
st_corss_attention = format_attention(outputs.cross_attentions)
layer_nums = st_corss_attention.shape[0] ## head
head_nums = st_corss_attention.shape[1] ## head


In [16]:
# test = format_attention(outputs.cross_attentions)
# import plotly.express as px

# fig = px.imshow(test.detach().numpy()[5][0], binary_string=False,
#                 labels=dict(x='Text token', y='Title token'), 
#                 x=encoder_text, y=decoder_text,
#                 color_continuous_scale='Blues', facet_row=)
# fig.update_xaxes(showticklabels=False)
# fig.update_yaxes(showticklabels=False)
# fig.show()

In [79]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

viz_layer = 5
head_nums = 2


In [88]:
layers = []
for layer in range(0, layer_nums) :
    layer_mat = st_corss_attention.detach().numpy()[layer]
    max_value, min_value = layer_mat.max(), layer_mat.min()
    sub_fig = make_subplots(head_nums, 1)

    for head in range(1, head_nums+1):
        head_mat = layer_mat[head-1]
        scale = [round(0.1*i,2) for i in range(11)]
        go_fig = go.Heatmap(
                        z=head_mat,
                        x=encoder_text,
                        y=decoder_text,
                        colorscale='Reds',
                        hoverongaps=False,
                        zmin=min_value,
                        zmax=max_value,
                        visible=True if layer == viz_layer else False)
        sub_fig.add_trace(go_fig, head, 1)
    sub_fig.update_xaxes(showticklabels=False)
    sub_fig.update_yaxes(showticklabels=False)

    sub_fig.update_layout(
        autosize=True,
        margin=dict(t=100, b=5, l=5, r=5),
        template="plotly_white")
    # Add dropdowns
    button_layer_1_height = 1.15
    sub_fig.update_layout(
        updatemenus=[ ## button 생성
            dict(
                buttons=list([
                    dict(
                        args=["colorscale", "Reds"],
                        label="Reds",
                        method="restyle"
                    ),
                    dict(
                        args=["colorscale", "Viridis"],
                        label="Viridis",
                        method="restyle"
                    ),
                    dict(
                        args=["colorscale", "Cividis"],
                        label="Cividis",
                        method="restyle"
                    ),
                ]),
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.32,
                xanchor="left",
                y=button_layer_1_height,
                yanchor="top"
            ),
            dict(
                buttons=list([
                    dict(
                        args=["reversescale", False],
                        label="False",
                        method="restyle"
                    ),
                    dict(
                        args=["reversescale", True],
                        label="True",
                        method="restyle"
                    )
                ]),
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.58,
                xanchor="left",
                y=button_layer_1_height,
                yanchor="top"
            ),
        ]
    )
    sub_fig.update_layout(
        annotations=[
            dict(text="colorscale", x=0.25, xref="paper", y=1.2, yref="paper",
                                 align="left", showarrow=False),
            dict(text="Reverse<br>Colorscale", x=0.54, xref="paper", y=1.21,
                                 yref="paper", showarrow=False),
        ])
    layers.append(sub_fig)

layers
# sub_fig.show()
# py.plot(fig, filename = 'heatmap', auto_open=True)

[Figure({
     'data': [{'colorscale': [[0.0, 'rgb(255,245,240)'], [0.125,
                              'rgb(254,224,210)'], [0.25, 'rgb(252,187,161)'],
                              [0.375, 'rgb(252,146,114)'], [0.5, 'rgb(251,106,74)'],
                              [0.625, 'rgb(239,59,44)'], [0.75, 'rgb(203,24,29)'],
                              [0.875, 'rgb(165,15,21)'], [1.0, 'rgb(103,0,13)']],
               'hoverongaps': False,
               'type': 'heatmap',
               'visible': False,
               'x': [본, ▁연구, 는, ▁최근, ▁20, 년간, ▁스웨덴, ▁지방자치단체, 가, ▁경험, 한, ▁주요, ▁환경,
                     정책을, ▁검토, 하여, ▁거, 버, 넌, 스의, ▁특징, 이, ▁어떻게, ▁변화, 하였, 는지, ▁분석, 하고,
                     ▁그에, ▁따른, ▁시사, 점을, ▁도출, 한, ▁것이다., ▁구체적인, ▁사례, 로, ▁지],
               'xaxis': 'x',
               'y': [스, 웨, 덴, ▁지역, 환경, 정책, ▁거, 버, 넌, 스의, ▁변화와, ▁특징, ▁연구, ▁-, 지방, 의,
                     제, 21, (L, A, 21, ),, ▁녹색, 성장, ▁지역, 투자, 프로그램, (L, IP, ),, ▁기후,
                     변화, 투자, 프로그램, (K, l, im, p, )의, 

In [94]:
updatemenus = list([
    dict(active=-1,
         buttons=list([
            dict(label = 'High',
                 method = 'update',
                 args = [{'visible': [True, True, False, False]},
                         {'title': 'Yahoo High',
                          'annotations': 0}]),
            dict(label = 'Low',
                 method = 'update',
                 args = [{'visible': [False, False, True, True]},
                         {'title': 'Yahoo Low',
                          'annotations': 1}]),
            dict(label = 'Both',
                 method = 'update',
                 args = [{'visible': [True, True, True, True]},
                         {'title': 'Yahoo',
                          'annotations': 2}]),
            dict(label = 'Reset',
                 method = 'update',
                 args = [{'visible': [True, False, True, False]},
                         {'title': 'Yahoo',
                          'annotations': []}])
        ]),
    )
])

In [104]:
buttons_contents = []
for target_layer in range(layers) :
    buttons_contents.append(
		dict(label = target_layer,
            method = 'update',
            args = [{'visible': [ True if i == target_layer else False for i in range(layers)]},
					{'annotations': []}])
        )
updatemenus=list([dict(active=-1, buttons=buttons_contents)])
updatemenus

[{'active': -1,
  'buttons': [{'label': 0,
    'method': 'update',
    'args': [{'visible': [True, False, False, False, False, False]},
     {'annotations': []}]},
   {'label': 1,
    'method': 'update',
    'args': [{'visible': [False, True, False, False, False, False]},
     {'annotations': []}]},
   {'label': 2,
    'method': 'update',
    'args': [{'visible': [False, False, True, False, False, False]},
     {'annotations': []}]},
   {'label': 3,
    'method': 'update',
    'args': [{'visible': [False, False, False, True, False, False]},
     {'annotations': []}]},
   {'label': 4,
    'method': 'update',
    'args': [{'visible': [False, False, False, False, True, False]},
     {'annotations': []}]},
   {'label': 5,
    'method': 'update',
    'args': [{'visible': [False, False, False, False, False, True]},
     {'annotations': []}]}]}]